In [1]:
# Importing libraries
import pandas as pd
import numpy as np

import datetime as dt
from datetime import date

import gc

#### Importing data on COVID-19 maintained by [Our World in Data](https://ourworldindata.org/coronavirus)
Data Citation:- Hasell, J., Mathieu, E., Beltekian, D. et al. A cross-country database of COVID-19 testing. Sci Data 7, 345 (2020). https://doi.org/10.1038/s41597-020-00688-8

In [3]:
df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv') #https://covid.ourworldindata.org/data/owid-covid-data.csv
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [4]:
# Looking for shape of data, data columns, and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78457 entries, 0 to 78456
Data columns (total 59 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   iso_code                               78457 non-null  object 
 1   continent                              74625 non-null  object 
 2   location                               78457 non-null  object 
 3   date                                   78457 non-null  object 
 4   total_cases                            76903 non-null  float64
 5   new_cases                              76901 non-null  float64
 6   new_cases_smoothed                     75900 non-null  float64
 7   total_deaths                           67514 non-null  float64
 8   new_deaths                             67672 non-null  float64
 9   new_deaths_smoothed                    75900 non-null  float64
 10  total_cases_per_million                76485 non-null  float64
 11  ne

### 1. Data Pre-processing

<b>1. Datetime datatype for date column<b/>

In [5]:
# Converting data column format from object to datetime
df.date=pd.to_datetime(df.date)
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month
# df['day'] = df['date'].dt.day

<b>2. Dropping duplicate rows if any<b/>

In [6]:
# Checking for duplicate rows
print(df.duplicated().sum())
df.drop_duplicates(inplace=True, ignore_index=True)

0


<b>3. Removing countries/locations with no data entry for confirmed cases<b/>

In [7]:
# Looking for countries with no single data on total cases and dropping them

print('Total Locations : {}'.format(str(df.location.nunique())))

null_countries=[]
for country in df['location'].unique():
    if df[df['location']==country]['total_cases'].notna().sum()==0:
        null_countries.append(country)
print('Null Locations :',null_countries)

df = df[~df.location.isin(null_countries)]

del null_countries

print('Total Locations :', df.location.nunique())

Total Locations : 215
Null Locations : ['Anguilla', 'Bermuda', 'Cayman Islands', 'Faeroe Islands', 'Falkland Islands', 'Gibraltar', 'Greenland', 'Guernsey', 'Hong Kong', 'Isle of Man', 'Jersey', 'Macao', 'Montserrat', 'Northern Cyprus', 'Saint Helena', 'Turks and Caicos Islands']
Total Locations : 199


<B>4. Removing unnecessary columns<b/>

In [8]:
cols = ['new_cases_smoothed','new_deaths_smoothed','new_cases_smoothed_per_million','new_deaths_smoothed_per_million','icu_patients',
        'icu_patients_per_million', 'hosp_patients','hosp_patients_per_million', 'weekly_icu_admissions','weekly_icu_admissions_per_million', 
        'weekly_hosp_admissions','weekly_hosp_admissions_per_million','new_tests', 'total_tests', 'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand', 'positive_rate', 'tests_per_case','tests_units', 
        'total_vaccinations','people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations',
       'new_vaccinations_smoothed', 'total_vaccinations_per_hundred','people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred',
       'new_vaccinations_smoothed_per_million', 'extreme_poverty','handwashing_facilities']

In [9]:
# Dropping unnecessary columns
df=df.drop(columns=cols,axis=1)
df.reset_index(drop=True, inplace=True)

del cols

<b>5. Dealing with null values<b/>

In [10]:
# df.info()

* Imputing null daily new cases and new deaths cells with zero:

In [11]:
# Filling zero values for new cases and new deaths columns

df['new_cases'].fillna(0, inplace=True)
df['new_deaths'].fillna(0,inplace=True)

* Imputing null values in total_cases and total_deaths column using below formuls:*

>* If previous row location equals current row location, imputing with addition of previous row total_cases (total_deaths) and current row new_cases (new_deaths) 

>* If it's a first entry for a location, imputing with new_cases (new_deaths) values


In [12]:
# Dealing with empty values in total cases and total deaths columns

# Restting index before dealing with df through index
df.reset_index(drop=True, inplace=True)

for index, row in df.iterrows():
    if pd.isnull(row['total_cases']):
        if index==0 or (df.iat[index,2]) != (df.iat[index-1,2]):
            df.iat[index,4]=df.iat[index,5]
        elif df.iat[index,2]==df.iat[index-1,2]:
            df.iat[index,4] = np.nansum([df.iat[index-1,4], df.iat[index,5]])
        
            
    if pd.isnull(row['total_deaths']):
        if index==0 or (df.iat[index,2]) != (df.iat[index-1,2]):
            df.iat[index,6]=df.iat[index,7]

        elif df.iat[index,2]==df.iat[index-1,2]:
            df.iat[index,6]= np.nansum([df.iat[index-1,6] + df.iat[index,7]])

In [13]:
# df.info()

* Probing into population and location data to deal with incidence rate values

In [14]:
df[df.population.isnull()]['location'].unique()

array(['International'], dtype=object)

In [15]:
# International data
intl = df[df['location']=='International'].copy()
intl.drop(columns=['continent'], inplace=True)
intl.reset_index(drop=True, inplace=True)

df = df[df.location!='International']
df.reset_index(drop=True, inplace=True)

del intl

In [16]:
# df.info()

* Calculating incidence rates

In [17]:
for i, row in df.iterrows():
    if pd.isnull(row['total_cases_per_million']):
        df.iat[i,8]=((df.iat[i,4]/df.iat[i,14])*1000000).round(3)
        
    if pd.isnull(row['new_cases_per_million']):
        df.iat[i,9]=((df.iat[i,5]/df.iat[i,14])*1000000).round(3)
        
    if pd.isnull(row['total_deaths_per_million']):
        df.iat[i,10]=((df.iat[i,6]/df.iat[i,14])*1000000).round(3)
        
    if pd.isnull(row['new_deaths_per_million']):
        df.iat[i,11]=((df.iat[i,7]/df.iat[i,14])*1000000).round(3)
    

In [18]:
# df.info()

* Confirming locations and segregating data for location wise time series data and continent wise aggregated time-series data

In [19]:
df['iso_code'].unique()

array(['AFG', 'OWID_AFR', 'ALB', 'DZA', 'AND', 'AGO', 'ATG', 'ARG', 'ARM',
       'OWID_ASI', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB', 'BLR',
       'BEL', 'BLZ', 'BEN', 'BTN', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN',
       'BGR', 'BFA', 'BDI', 'KHM', 'CMR', 'CAN', 'CPV', 'CAF', 'TCD',
       'CHL', 'CHN', 'COL', 'COM', 'COG', 'CRI', 'CIV', 'HRV', 'CUB',
       'CYP', 'CZE', 'COD', 'DNK', 'DJI', 'DMA', 'DOM', 'ECU', 'EGY',
       'SLV', 'GNQ', 'ERI', 'EST', 'SWZ', 'ETH', 'OWID_EUR', 'OWID_EUN',
       'FJI', 'FIN', 'FRA', 'GAB', 'GMB', 'GEO', 'DEU', 'GHA', 'GRC',
       'GRD', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'HUN', 'ISL',
       'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 'ITA', 'JAM', 'JPN',
       'JOR', 'KAZ', 'KEN', 'OWID_KOS', 'KWT', 'KGZ', 'LAO', 'LVA', 'LBN',
       'LSO', 'LBR', 'LBY', 'LIE', 'LTU', 'LUX', 'MDG', 'MWI', 'MYS',
       'MDV', 'MLI', 'MLT', 'MHL', 'MRT', 'MUS', 'MEX', 'FSM', 'MDA',
       'MCO', 'MNG', 'MNE', 'MAR', 'MOZ', 'MMR', 'NAM', 'NPL', 'NLD',
  

In [20]:
df['continent'].unique()

array(['Asia', nan, 'Europe', 'Africa', 'North America', 'South America',
       'Oceania'], dtype=object)

In [21]:
df[df['continent'].isnull()]['iso_code'].unique()

array(['OWID_AFR', 'OWID_ASI', 'OWID_EUR', 'OWID_EUN', 'OWID_NAM',
       'OWID_OCE', 'OWID_SAM', 'OWID_WRL'], dtype=object)

In [22]:
print(df[df['iso_code']=='OWID_KOS']['continent'].unique())
print(df[df['iso_code']=='OWID_KOS']['location'].unique())

['Europe']
['Kosovo']


Source: Wikipedia
<br>
> Kosovo officially the Republic of Kosovo is a partially-recognised state and disputed territory in Southeastern Europe.

In [23]:
iso = ['OWID_AFR','OWID_ASI','OWID_EUR','OWID_EUN',
       'OWID_INT','OWID_NAM','OWID_OCE','OWID_SAM','OWID_WRL']
df[df['iso_code'].isin(iso)]['continent'].unique()

array([nan], dtype=object)

In [24]:
df[df['iso_code'].isin(iso)]['location'].unique()

array(['Africa', 'Asia', 'Europe', 'European Union', 'North America',
       'Oceania', 'South America', 'World'], dtype=object)

In [25]:
continents = ['Africa', 'Asia', 'Europe','North America', 'Oceania', 'South America']

In [26]:
# Data on world
world = df[df['location']=='World'].copy()
world.drop(columns=['continent'], inplace=True)
world.reset_index(drop=True, inplace=True)

# Continent-specific data
conti = df[df['location'].isin(continents)].copy()
conti.drop(columns=['continent'], inplace=True)
conti.reset_index(drop=True, inplace=True)

# European-Union specific data
eu_un = df[df['location']=='European Union'].copy()
eu_un.drop(columns=['continent'], inplace=True)
eu_un.reset_index(drop=True, inplace=True)

In [27]:
world.info()
# conti.info()
# eu_un.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   iso_code                    434 non-null    object        
 1   location                    434 non-null    object        
 2   date                        434 non-null    datetime64[ns]
 3   total_cases                 434 non-null    float64       
 4   new_cases                   434 non-null    float64       
 5   total_deaths                434 non-null    float64       
 6   new_deaths                  434 non-null    float64       
 7   total_cases_per_million     434 non-null    float64       
 8   new_cases_per_million       434 non-null    float64       
 9   total_deaths_per_million    434 non-null    float64       
 10  new_deaths_per_million      434 non-null    float64       
 11  reproduction_rate           432 non-null    float64       

In [28]:
cols = ['reproduction_rate', 'stringency_index','population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'cardiovasc_death_rate','diabetes_prevalence', 'female_smokers', 'male_smokers',
       'hospital_beds_per_thousand', 'life_expectancy','human_development_index']

conti.drop(columns=cols, inplace=True)
conti.reset_index(drop=True, inplace=True)

eu_un.drop(columns=cols, inplace=True)
eu_un.reset_index(drop=True, inplace=True)

In [29]:
df=df[~df['iso_code'].isin(iso)]
df.reset_index(inplace=True,drop=True)

del continents, iso, cols
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73613 entries, 0 to 73612
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   iso_code                    73613 non-null  object        
 1   continent                   73613 non-null  object        
 2   location                    73613 non-null  object        
 3   date                        73613 non-null  datetime64[ns]
 4   total_cases                 73613 non-null  float64       
 5   new_cases                   73613 non-null  float64       
 6   total_deaths                73613 non-null  float64       
 7   new_deaths                  73613 non-null  float64       
 8   total_cases_per_million     73613 non-null  float64       
 9   new_cases_per_million       73613 non-null  float64       
 10  total_deaths_per_million    73613 non-null  float64       
 11  new_deaths_per_million      73613 non-null  float64   

In [30]:
df['location'].nunique()

190

In [31]:
gc.collect()

44

* Separate dataset for country specific indicatiors and updated COVID-19 current status

In [32]:
i = ['population', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'hospital_beds_per_thousand', 'life_expectancy',
       'human_development_index']

countries = df.groupby(by=['location'], as_index=False)[i].max()

df.drop(columns=i, inplace=True)
df.reset_index(drop=True, inplace=True)

df1=df[df['date']==df.date.max()].copy()

countries = countries.merge(df1, how='left', on='location')

del i

* Creating a column for Case Fatality Rate calculated by:
>(Total Deaths per Million / Total Cases Per Million) * 100

In [33]:
df['cfr']=(df['total_deaths_per_million']/df['total_cases_per_million']) * 100

In [34]:
countries['cfr']=(countries['total_deaths_per_million']/countries['total_cases_per_million']) * 100

In [35]:
world['cfr']=(world['total_deaths_per_million']/world['total_cases_per_million']) * 100

In [36]:
df.to_csv('all.csv', index=False)
world.to_csv('world.csv', index=False)
conti.to_csv('continents.csv', index=False)
countries.to_csv('parameters.csv', index=False)